In [28]:
import pandas as pd
import numpy as np

### Load in Cleaned Data

- Discuss Preprocessing and cleaning

### Market Orders (Trades)

In [29]:
trades=pd.read_csv('data/cleaned_trades.csv',index_col=0)


In [45]:
simple_trades=trades[['Exchange','Symbol','Trade_Volume','Trade_Price']].head(1000).copy()
simple_trades

,Exchange,Symbol,Trade_Volume,Trade_Price
2020-01-15 09:30:01.004096,N,AAPL,100,311.94
2020-01-15 09:30:01.100463,N,AAPL,3,311.91
2020-01-15 09:30:01.438061,N,AAPL,10,311.89
2020-01-15 09:30:01.438068,N,AAPL,46,311.89
2020-01-15 09:30:01.677976,N,AAPL,1,311.95
...,...,...,...,...
2020-01-15 10:00:52.113187,N,AAPL,200,315.08
2020-01-15 10:00:53.835758,N,AAPL,10,315.00
2020-01-15 10:00:53.849823,N,AAPL,200,314.95
2020-01-15 10:00:53.865426,N,AAPL,100,315.00


### Limit Orders 

In [39]:
quotes=pd.read_csv('data/cleaned_quotes.csv',index_col=0)


simple_quotes=quotes[['Exchange','Symbol','Bid_Price','Bid_Size','Offer_Price','Offer_Size']].head(1000).copy()
simple_quotes

,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size
2020-01-15 09:30:00.953633,N,AAPL,311.94,1.0,326.18,1.0
2020-01-15 09:30:01.004096,N,AAPL,310.85,1.0,326.18,1.0
2020-01-15 09:30:01.076382,N,AAPL,310.85,1.0,311.96,1.0
2020-01-15 09:30:01.645062,N,AAPL,310.85,1.0,311.95,4.0
2020-01-15 09:30:01.677976,N,AAPL,310.85,1.0,311.95,3.0
...,...,...,...,...,...,...
2020-01-15 09:32:49.059054,N,AAPL,312.97,1.0,313.06,2.0
2020-01-15 09:32:49.088780,N,AAPL,312.96,3.0,313.06,2.0
2020-01-15 09:32:49.487021,N,AAPL,312.98,2.0,313.07,2.0
2020-01-15 09:32:49.507219,N,AAPL,312.96,1.0,313.07,2.0


### Label Direction of Trades

- Utilize Lee-Ready Tick Test for Trade Classification
- Need to review paper and methodology

In [49]:
def label_trade(x,quote_data):
    last_quote=quote_data[quote_data.index<x.index][-1]
  
    last_bid=last_quote['Bid_Price']
    last_ask=last_quote['Offer_Price']

    if x['Trade_Price']<=last_bid:
        return -1
    elif x['Trade_Price']>=last_ask:
        return 1
    else:
        return 0

simple_trades.apply(lambda x: label_trade(x,simple_quotes),axis=1)


ValueError: Lengths must match to compare

### Label our States

- Spread Tightens or Widens compared to last BBO
- Top of Book Relation to last BBO

In [42]:
simple_quotes['spread']=simple_quotes['Offer_Price']-simple_quotes['Bid_Price']
simple_quotes['last_spread']=simple_quotes['spread'].shift(1)

#label state as 1 or -1 based on widen or tighten
def label_spread(x):
    if x['spread']>x['last_spread']:
        # spread widens
        return 1
    elif x['spread']<x['last_spread']:
        # spread tightens 
        return -1
    else:
        return 0
simple_quotes['spread_state']=simple_quotes.apply(lambda x:label_spread(x),axis=1)


### label relation of top levels
simple_quotes['last_Bid_Price']=simple_quotes['Bid_Price'].shift(1)
simple_quotes['last_Offer_Price']=simple_quotes['Offer_Price'].shift(1)

def label_top_book(x):
    if x['Bid_Price']<x['last_Bid_Price']:
        # top bid decreases
        return 1
    elif x['Bid_Price']>x['last_Bid_Price']:
        # top bid increases 
        return -1
    elif x['Offer_Price']>x['last_Offer_Price']:
        # top ask increases
        return 2
    elif x['Offer_Price']<x['last_Offer_Price']:
        # top ask decreases 
        return -2
    else:
        # no difference
        return 0

simple_quotes['book_state']=simple_quotes.apply(lambda x: label_top_book(x), axis=1)

simple_quotes=simple_quotes.drop(columns=['last_spread','last_Bid_Price', 'last_Offer_Price'])

# embed order flow data; last N seconds of trades






,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,spread,spread_state,book_state
2020-01-15 09:30:00.953633,N,AAPL,311.94,1.0,326.18,1.0,14.24,0,0
2020-01-15 09:30:01.004096,N,AAPL,310.85,1.0,326.18,1.0,15.33,1,1
2020-01-15 09:30:01.076382,N,AAPL,310.85,1.0,311.96,1.0,1.11,-1,-2
2020-01-15 09:30:01.645062,N,AAPL,310.85,1.0,311.95,4.0,1.10,-1,-2
2020-01-15 09:30:01.677976,N,AAPL,310.85,1.0,311.95,3.0,1.10,0,0
...,...,...,...,...,...,...,...,...,...
2020-01-15 09:32:49.059054,N,AAPL,312.97,1.0,313.06,2.0,0.09,-1,-1
2020-01-15 09:32:49.088780,N,AAPL,312.96,3.0,313.06,2.0,0.10,1,1
2020-01-15 09:32:49.487021,N,AAPL,312.98,2.0,313.07,2.0,0.09,-1,-1
2020-01-15 09:32:49.507219,N,AAPL,312.96,1.0,313.07,2.0,0.11,1,1
